In [11]:
%%writefile yolo_baggage_detection.py
import os
import urllib.request
import zipfile
import argparse
import yaml
import subprocess
import glob
import shutil
from pathlib import Path

class RoboflowDatasetDownloader:
    def __init__(self, api_key=None):
        """
        Initialize the Roboflow dataset downloader

        Args:
            api_key: Roboflow API key (optional)
        """
        self.api_key = api_key
        self.datasets = {
            "test_ds": {
                "url": "https://universe.roboflow.com/romain-dreuilhet/test_ds/dataset/1",
                "download_url": "https://universe.roboflow.com/ds/c55EGRQxaK?key=87PNQIVz5x",
                "format": "yolov8"
            },
            "1_baghandle": {
                "url": "https://universe.roboflow.com/project1bag-and-handle-detection/1-baghandle/dataset/9",
                "download_url": "https://universe.roboflow.com/ds/3zNkrF3Huw?key=XhBucC2NHd",
                "format": "yolov8"
            },
            "shopping": {
                "url": "https://universe.roboflow.com/testworking/shopping-3p7xz/dataset/3",
                "download_url": "https://universe.roboflow.com/ds/Ui3IwDM8Iy?key=SleB2d9LnX",
                "format": "yolov8"
            }
        }

    def download_dataset(self, dataset_name, api_key=None, output_dir="datasets"):
        """
        Download a dataset from Roboflow

        Args:
            dataset_name: Name of the dataset to download
            api_key: Roboflow API key
            output_dir: Directory to save the downloaded dataset

        Returns:
            Path to the downloaded dataset
        """
        if api_key is None:
            api_key = self.api_key

        if api_key is None:
            raise ValueError("API key is required to download Roboflow datasets")

        if dataset_name not in self.datasets:
            raise ValueError(f"Dataset {dataset_name} not found. Available datasets: {list(self.datasets.keys())}")

        # Create output directory
        os.makedirs(output_dir, exist_ok=True)
        dataset_dir = os.path.join(output_dir, dataset_name)
        os.makedirs(dataset_dir, exist_ok=True)

        # Download the dataset
        download_url = self.datasets[dataset_name]["download_url"].format(api_key)
        zip_path = os.path.join(dataset_dir, f"{dataset_name}.zip")

        print(f"Downloading {dataset_name} dataset...")
        urllib.request.urlretrieve(download_url, zip_path)

        # Extract the dataset
        print(f"Extracting {dataset_name} dataset...")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(dataset_dir)

        # Remove the zip file
        os.remove(zip_path)

        print(f"Dataset {dataset_name} downloaded and extracted to {dataset_dir}")
        return dataset_dir

    def download_all_datasets(self, api_key=None, output_dir="datasets"):
        """
        Download all datasets

        Args:
            api_key: Roboflow API key
            output_dir: Directory to save the downloaded datasets

        Returns:
            Dictionary containing paths to all downloaded datasets
        """
        all_datasets = {}

        for dataset_name in self.datasets.keys():
            print(f"Downloading dataset: {dataset_name}")

            # Download the dataset
            dataset_dir = self.download_dataset(dataset_name, api_key, output_dir)
            all_datasets[dataset_name] = dataset_dir

        return all_datasets

    def merge_datasets(self, datasets_dirs, output_dir="merged_dataset"):
        """
        Merge multiple YOLOv5 datasets into a single dataset

        Args:
            datasets_dirs: Dictionary of paths to individual datasets
            output_dir: Directory to save the merged dataset

        Returns:
            Path to the merged dataset
        """
        # Create output directory
        os.makedirs(output_dir, exist_ok=True)

        # Create directories for train, val, and test sets
        for split in ['train', 'valid', 'test']:
            os.makedirs(os.path.join(output_dir, split, 'images'), exist_ok=True)
            os.makedirs(os.path.join(output_dir, split, 'labels'), exist_ok=True)

        # Load class mappings from each dataset
        all_classes = set()
        dataset_classes = {}

        for dataset_name, dataset_dir in datasets_dirs.items():
            yaml_path = os.path.join(dataset_dir, 'data.yaml')
            if os.path.exists(yaml_path):
                with open(yaml_path, 'r') as f:
                    dataset_yaml = yaml.safe_load(f)
                    dataset_classes[dataset_name] = dataset_yaml.get('names', [])
                    all_classes.update(dataset_classes[dataset_name])

        # Create a unified class mapping
        unified_classes = sorted(list(all_classes))
        class_mapping = {dataset_name: {old_idx: unified_classes.index(class_name)
                                       for old_idx, class_name in enumerate(classes)}
                        for dataset_name, classes in dataset_classes.items()}

        # Copy and remap data from each dataset
        for dataset_name, dataset_dir in datasets_dirs.items():
            dataset_mapping = class_mapping.get(dataset_name, {})

            for split in ['train', 'valid', 'test']:
                split_dir = os.path.join(dataset_dir, split)
                if not os.path.exists(split_dir):
                    continue

                # Copy images
                images_dir = os.path.join(split_dir, 'images')
                if os.path.exists(images_dir):
                    for img_path in glob.glob(os.path.join(images_dir, '*.*')):
                        img_name = os.path.basename(img_path)
                        # Add dataset prefix to avoid name conflicts
                        new_img_name = f"{dataset_name}_{img_name}"
                        shutil.copy2(img_path, os.path.join(output_dir, split, 'images', new_img_name))

                # Copy and remap labels
                labels_dir = os.path.join(split_dir, 'labels')
                if os.path.exists(labels_dir):
                    for label_path in glob.glob(os.path.join(labels_dir, '*.txt')):
                        label_name = os.path.basename(label_path)
                        img_name = os.path.splitext(label_name)[0]
                        # Add dataset prefix to match the image
                        new_label_name = f"{dataset_name}_{label_name}"

                        # Read and remap class indices
                        with open(label_path, 'r') as f_in:
                            lines = f_in.readlines()

                        remapped_lines = []
                        for line in lines:
                            parts = line.strip().split()
                            if len(parts) >= 5:  # class_id x_center y_center width height
                                old_class_id = int(parts[0])
                                # Remap class id to unified class id
                                new_class_id = dataset_mapping.get(old_class_id, old_class_id)
                                parts[0] = str(new_class_id)
                                remapped_lines.append(' '.join(parts) + '\n')
                            else:
                                remapped_lines.append(line)

                        # Write remapped labels
                        with open(os.path.join(output_dir, split, 'labels', new_label_name), 'w') as f_out:
                            f_out.writelines(remapped_lines)

        # Create merged data.yaml
        yaml_data = {
            'path': os.path.abspath(output_dir),
            'train': os.path.join('train', 'images'),
            'val': os.path.join('valid', 'images'),
            'test': os.path.join('test', 'images'),
            'names': unified_classes,
            'nc': len(unified_classes)
        }

        with open(os.path.join(output_dir, 'data.yaml'), 'w') as f:
            yaml.dump(yaml_data, f, default_flow_style=False)

        print(f"Merged dataset created at {output_dir}")
        print(f"Unified classes: {unified_classes}")
        return output_dir

def setup_yolov5(yolov5_dir="yolov5"):
    """
    Clone YOLOv5 repository and install requirements

    Args:
        yolov5_dir: Directory to clone YOLOv5 into

    Returns:
        Path to the YOLOv5 repository
    """
    if not os.path.exists(yolov5_dir):
        print("Cloning YOLOv5 repository...")
        subprocess.run([
            "git", "clone", "https://github.com/ultralytics/yolov5.git", yolov5_dir
        ], check=True)
    else:
        print("YOLOv5 repository already exists.")

    # Install requirements
    req_file = os.path.join(yolov5_dir, "requirements.txt")
    if os.path.exists(req_file):
        print("Installing YOLOv5 requirements...")
        subprocess.run([
            "pip", "install", "-r", req_file
        ], check=True)

    return yolov5_dir

def train_yolov5(yolov5_dir, data_yaml, model_size="s", epochs=100, batch_size=16, img_size=640, output_dir="runs/train"):
    """
    Train YOLOv5 on the specified dataset

    Args:
        yolov5_dir: Path to YOLOv5 repository
        data_yaml: Path to data.yaml file
        model_size: YOLOv5 model size (n, s, m, l, x)
        epochs: Number of training epochs
        batch_size: Batch size
        img_size: Image size for training
        output_dir: Directory to save training results

    Returns:
        Path to the trained model weights
    """
    print(f"Training YOLOv5{model_size} on {data_yaml}...")

    # Run training command
    subprocess.run([
        "python", os.path.join(yolov5_dir, "train.py"),
        "--data", data_yaml,
        "--weights", f"yolov5{model_size}.pt",
        "--epochs", str(epochs),
        "--batch-size", str(batch_size),
        "--img", str(img_size),
        "--project", os.path.dirname(output_dir),
        "--name", os.path.basename(output_dir)
    ], check=True)

    # Get the path to the best weights
    weights_path = os.path.join(output_dir, "weights", "best.pt")
    if os.path.exists(weights_path):
        print(f"Training completed. Best weights saved to {weights_path}")
    else:
        print("Training completed, but could not find best weights.")
        # Try to find weights in case the output structure differs
        weights_path = list(Path(output_dir).rglob("best.pt"))
        if weights_path:
            weights_path = str(weights_path[0])
            print(f"Found weights at {weights_path}")
        else:
            weights_path = None

    return weights_path

def export_to_pb(yolov5_dir, weights_path, output_dir="exported_models", img_size=640):
    """
    Export YOLOv5 model to .pb format

    Args:
        yolov5_dir: Path to YOLOv5 repository
        weights_path: Path to trained weights (.pt file)
        output_dir: Directory to save exported model
        img_size: Image size for the model

    Returns:
        Path to the exported model
    """
    print(f"Exporting model from {weights_path} to TensorFlow SavedModel format...")

    # Create output directory
    os.makedirs(output_dir, exist_ok=True)

    # Run export command
    subprocess.run([
        "python", os.path.join(yolov5_dir, "export.py"),
        "--weights", weights_path,
        "--include", "saved_model",
        "--img", str(img_size),
        "--batch", "1",
        "--simplify"
    ], check=True)

    # The exported model should be in the same directory as the weights but with a different name
    weights_dir = os.path.dirname(weights_path)
    exported_model = os.path.join(weights_dir, os.path.splitext(os.path.basename(weights_path))[0] + "_saved_model")

    if os.path.exists(exported_model):
        # Copy to the specified output directory
        target_path = os.path.join(output_dir, "yolov5_saved_model")
        if os.path.exists(target_path):
            shutil.rmtree(target_path)
        shutil.copytree(exported_model, target_path)
        print(f"Model exported to {target_path}")
        return target_path
    else:
        print("Could not find exported model.")
        return None

def main():
    parser = argparse.ArgumentParser(description="Train YOLOv5 on Roboflow datasets")
    parser.add_argument("--api-key", type=str, required=True, help="Roboflow API key")
    parser.add_argument("--output-dir", type=str, default="data", help="Directory to save datasets and models")
    parser.add_argument("--model-size", type=str, default="s", choices=["n", "s", "m", "l", "x"],
                        help="YOLOv5 model size (n=nano, s=small, m=medium, l=large, x=xlarge)")
    parser.add_argument("--epochs", type=int, default=100, help="Number of training epochs")
    parser.add_argument("--batch-size", type=int, default=16, help="Batch size for training")
    parser.add_argument("--img-size", type=int, default=640, help="Image size for training")
    parser.add_argument("--mode", type=str, default="merged", choices=["merged", "individual"],
                        help="Train on merged dataset or individual datasets")
    parser.add_argument("--datasets", type=str, nargs="+", default=["test_ds", "1_baghandle", "shopping"],
                        help="Datasets to download and train on")

    args = parser.parse_args()

    # Create output directories
    datasets_dir = os.path.join(args.output_dir, "datasets")
    models_dir = os.path.join(args.output_dir, "models")
    os.makedirs(datasets_dir, exist_ok=True)
    os.makedirs(models_dir, exist_ok=True)

    # Download datasets
    downloader = RoboflowDatasetDownloader(api_key=args.api_key)
    dataset_paths = {}

    for dataset_name in args.datasets:
        dataset_path = downloader.download_dataset(dataset_name, args.api_key, datasets_dir)
        dataset_paths[dataset_name] = dataset_path

    # Setup YOLOv5
    yolov5_dir = setup_yolov5()

    if args.mode == "merged":
        # Merge datasets
        merged_dir = downloader.merge_datasets(dataset_paths, os.path.join(datasets_dir, "merged"))
        data_yaml = os.path.join(merged_dir, "data.yaml")

        # Train on merged dataset
        output_dir = os.path.join(models_dir, "merged")
        weights_path = train_yolov5(
            yolov5_dir=yolov5_dir,
            data_yaml=data_yaml,
            model_size=args.model_size,
            epochs=args.epochs,
            batch_size=args.batch_size,
            img_size=args.img_size,
            output_dir=output_dir
        )

        # Export model
        if weights_path:
            export_to_pb(
                yolov5_dir=yolov5_dir,
                weights_path=weights_path,
                output_dir=os.path.join(models_dir, "exported"),
                img_size=args.img_size
            )
    else:
        # Train on individual datasets
        for dataset_name, dataset_path in dataset_paths.items():
            data_yaml = os.path.join(dataset_path, "data.yaml")
            if os.path.exists(data_yaml):
                output_dir = os.path.join(models_dir, dataset_name)

                # Train
                weights_path = train_yolov5(
                    yolov5_dir=yolov5_dir,
                    data_yaml=data_yaml,
                    model_size=args.model_size,
                    epochs=args.epochs,
                    batch_size=args.batch_size,
                    img_size=args.img_size,
                    output_dir=output_dir
                )

                # Export model
                if weights_path:
                    export_to_pb(
                        yolov5_dir=yolov5_dir,
                        weights_path=weights_path,
                        output_dir=os.path.join(models_dir, "exported", dataset_name),
                        img_size=args.img_size
                    )

    print("Training and export completed!")

if __name__ == "__main__":
    main()

Overwriting yolo_baggage_detection.py


In [12]:
%%writefile yolo_inference.py
import os
import argparse
import cv2
import torch
import numpy as np
import glob
from pathlib import Path

def run_inference(model_path, image_path, output_dir="outputs", img_size=340, conf_threshold=0.25):
    """
    Run inference with YOLOv5 model on an image

    Args:
        model_path: Path to YOLOv5 weights (.pt file)
        image_path: Path to input image
        output_dir: Directory to save output image
        img_size: Image size for the model
        conf_threshold: Confidence threshold for detections

    Returns:
        Path to output image
    """
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)

    # Load the model
    model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path)
    model.conf = conf_threshold  # Set confidence threshold
    model.iou = 0.45  # Set IoU threshold
    model.img = img_size  # Set inference size

    # Run inference
    results = model(image_path)

    # Save results
    output_path = os.path.join(output_dir, os.path.basename(image_path))
    results.save(save_dir=output_dir)

    # Also save the raw prediction data for further analysis
    results_dict = results.pandas().xyxy[0].to_dict(orient='records')

    return output_path, results_dict

def process_directory(model_path, input_dir, output_dir="outputs", img_size=640, conf_threshold=0.25):
    """
    Process all images in a directory with YOLOv5 model

    Args:
        model_path: Path to YOLOv5 weights (.pt file)
        input_dir: Directory containing input images
        output_dir: Directory to save output images
        img_size: Image size for the model
        conf_threshold: Confidence threshold for detections

    Returns:
        Number of processed images
    """
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)

    # Load the model
    model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path)
    model.conf = conf_threshold  # Set confidence threshold
    model.iou = 0.45  # Set IoU threshold
    model.img = img_size  # Set inference size

    # Get all image files
    image_files = []
    for ext in ['jpg', 'jpeg', 'png', 'bmp']:
        image_files.extend(glob.glob(os.path.join(input_dir, f"*.{ext}")))
        image_files.extend(glob.glob(os.path.join(input_dir, f"*.{ext.upper()}")))

    # Process each image
    for img_path in image_files:
        print(f"Processing {img_path}...")
        results = model(img_path)

        # Get base filename
        base_name = os.path.basename(img_path)

        # Save results
        results.save(save_dir=output_dir)

    return len(image_files)

def main():
    parser = argparse.ArgumentParser(description="Run inference with trained YOLOv5 model")
    parser.add_argument("--model", type=str, required=True, help="Path to trained model weights (.pt file)")
    parser.add_argument("--image", type=str, help="Path to input image")
    parser.add_argument("--dir", type=str, help="Directory containing input images")
    parser.add_argument("--output", type=str, default="outputs", help="Directory to save output images")
    parser.add_argument("--img-size", type=int, default=640, help="Image size for inference")
    parser.add_argument("--conf", type=float, default=0.25, help="Confidence threshold for detections")

    args = parser.parse_args()

    if not args.image and not args.dir:
        parser.error("Either --image or --dir must be specified")

    if args.image and args.dir:
        parser.error("Only one of --image or --dir can be specified")

    # Create output directory
    os.makedirs(args.output, exist_ok=True)

    if args.image:
        # Process a single image
        if not os.path.exists(args.image):
            print(f"Error: Image not found: {args.image}")
            return

        output_path, results = run_inference(
            model_path=args.model,
            image_path=args.image,
            output_dir=args.output,
            img_size=args.img_size,
            conf_threshold=args.conf
        )

        print(f"Inference completed. Output saved to {output_path}")
        print(f"Detected objects: {results}")
    else:
        # Process a directory of images
        if not os.path.exists(args.dir):
            print(f"Error: Directory not found: {args.dir}")
            return

        num_processed = process_directory(
            model_path=args.model,
            input_dir=args.dir,
            output_dir=args.output,
            img_size=args.img_size,
            conf_threshold=args.conf
        )

        print(f"Inference completed. Processed {num_processed} images. Outputs saved to {args.output}")

if __name__ == "__main__":
    main()

Overwriting yolo_inference.py


In [13]:
%%writefile export_to_pb.py
import os
import argparse
import subprocess
import shutil
import tensorflow as tf
from pathlib import Path

def export_to_tf_saved_model(yolov5_dir, weights_path, output_dir, img_size=640):
    """
    Export YOLOv5 model to TensorFlow SavedModel format

    Args:
        yolov5_dir: Path to YOLOv5 repository
        weights_path: Path to trained weights (.pt file)
        output_dir: Directory to save exported model
        img_size: Image size for the model

    Returns:
        Path to the exported model
    """
    print(f"Exporting model from {weights_path} to TensorFlow SavedModel format...")

    # Create output directory
    os.makedirs(output_dir, exist_ok=True)

    # Ensure we have the export script
    if not os.path.exists(yolov5_dir):
        print("Cloning YOLOv5 repository...")
        subprocess.run([
            "git", "clone", "https://github.com/ultralytics/yolov5.git", yolov5_dir
        ], check=True)

    # Run export command
    subprocess.run([
        "python", os.path.join(yolov5_dir, "export.py"),
        "--weights", weights_path,
        "--include", "saved_model",
        "--img", str(img_size),
        "--batch", "1",
        "--simplify"
    ], check=True)

    # Find exported model
    weights_dir = os.path.dirname(weights_path)
    model_name = os.path.splitext(os.path.basename(weights_path))[0]
    exported_model = os.path.join(weights_dir, f"{model_name}_saved_model")

    if os.path.exists(exported_model):
        # Copy to the specified output directory
        target_path = os.path.join(output_dir, f"{model_name}_saved_model")
        if os.path.exists(target_path):
            shutil.rmtree(target_path)
        shutil.copytree(exported_model, target_path)
        print(f"Model exported to {target_path}")
        return target_path
    else:
        print("Could not find exported SavedModel directory.")
        # Try to find any SavedModel directory
        saved_models = list(Path(weights_dir).rglob("saved_model.pb"))
        if saved_models:
            saved_model_dir = saved_models[0].parent
            target_path = os.path.join(output_dir, f"{model_name}_saved_model")
            if os.path.exists(target_path):
                shutil.rmtree(target_path)
            shutil.copytree(saved_model_dir, target_path)
            print(f"Found and copied SavedModel to {target_path}")
            return target_path
        return None

def convert_saved_model_to_pb(saved_model_dir, output_dir, output_filename="model.pb"):
    """
    Convert TensorFlow SavedModel to .pb file

    Args:
        saved_model_dir: Path to SavedModel directory
        output_dir: Directory to save .pb file
        output_filename: Name of output .pb file

    Returns:
        Path to the output .pb file
    """
    print(f"Converting SavedModel from {saved_model_dir} to .pb format...")

    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, output_filename)

    # Load the SavedModel
    model = tf.saved_model.load(saved_model_dir)

    # Get concrete function
    concrete_func = model.signatures[tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY]

    # Convert to frozen graph
    frozen_func = tf.python.framework.convert_to_constants.convert_variables_to_constants_v2(concrete_func)
    frozen_func.graph.as_graph_def()

    # Save the frozen graph to a file
    tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                      logdir=output_dir,
                      name=output_filename,
                      as_text=False)

    print(f"Model converted to .pb format and saved to {output_path}")
    return output_path

def main():
    parser = argparse.ArgumentParser(description="Export YOLOv5 model to .pb format")
    parser.add_argument("--weights", type=str, required=True, help="Path to trained weights (.pt file)")
    parser.add_argument("--output-dir", type=str, default="exported_models", help="Directory to save exported model")
    parser.add_argument("--yolov5-dir", type=str, default="yolov5", help="Path to YOLOv5 repository")
    parser.add_argument("--img-size", type=int, default=640, help="Image size for the model")
    parser.add_argument("--output-filename", type=str, default="yolov5_model.pb", help="Name of output .pb file")

    args = parser.parse_args()

    # Check if weights file exists
    if not os.path.exists(args.weights):
        print(f"Error: Weights file not found: {args.weights}")
        return

    # Export to SavedModel
    saved_model_path = export_to_tf_saved_model(
        yolov5_dir=args.yolov5_dir,
        weights_path=args.weights,
        output_dir=args.output_dir,
        img_size=args.img_size
    )

    if saved_model_path:
        # Convert SavedModel to .pb
        pb_path = convert_saved_model_to_pb(
            saved_model_dir=saved_model_path,
            output_dir=args.output_dir,
            output_filename=args.output_filename
        )

        if pb_path:
            print(f"Export completed. Model exported to:")
            print(f"  - SavedModel: {saved_model_path}")
            print(f"  - .pb file: {pb_path}")
    else:
        print("Failed to export model to SavedModel format.")

if __name__ == "__main__":
    main()

Overwriting export_to_pb.py


In [14]:
from google.colab import drive
drive.mount('/content/drive')

# Create directories for output
!mkdir -p /content/drive/MyDrive/Edge_AI_CW/YOLO_Training/baggage_detection

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
# Create directories
!mkdir -p /content/drive/MyDrive/Edge_AI_CW/YOLO_Training/baggage_detection/datasets
!mkdir -p /content/drive/MyDrive/Edge_AI_CW/YOLO_Training/baggage_detection/models

In [7]:
%%writefile check_datasets.py
import os
import argparse

def main():
    parser = argparse.ArgumentParser(description="Check if datasets exist")
    parser.add_argument("--datasets-dir", type=str, required=True, help="Directory to check for datasets")

    args = parser.parse_args()

    datasets = ["test_ds", "1_baghandle", "shopping"]
    missing_datasets = []

    for dataset in datasets:
        dataset_path = os.path.join(args.datasets_dir, dataset)
        if not os.path.exists(dataset_path) or not os.path.exists(os.path.join(dataset_path, "data.yaml")):
            missing_datasets.append(dataset)

    if missing_datasets:
        print(f"Missing datasets: {missing_datasets}")
        return 1
    else:
        print("All datasets exist!")
        return 0

if __name__ == "__main__":
    exit(main())

Writing check_datasets.py


In [8]:
# Check if datasets already exist
!python check_datasets.py --datasets-dir /content/drive/MyDrive/Edge_AI_CW/YOLO_Training/baggage_detection/datasets

Missing datasets: ['test_ds', '1_baghandle', 'shopping']


In [9]:
ROBOFLOW_API_KEY = "rf_i8mhHw5LrlSElDtF1BrrShSRTRi1"

# Run training with specific paths
!python yolo_baggage_detection.py \
  --api-key $ROBOFLOW_API_KEY \
  --mode merged \
  --output-dir /content/drive/MyDrive/Edge_AI_CW/YOLO_Training/baggage_detection \
  --model-size s \
  --epochs 25

Extracting test_ds dataset...
Dataset test_ds downloaded and extracted to /content/drive/MyDrive/Edge_AI_CW/YOLO_Training/baggage_detection/datasets/test_ds
Extracting 1_baghandle dataset...
Dataset 1_baghandle downloaded and extracted to /content/drive/MyDrive/Edge_AI_CW/YOLO_Training/baggage_detection/datasets/1_baghandle
Extracting shopping dataset...
Dataset shopping downloaded and extracted to /content/drive/MyDrive/Edge_AI_CW/YOLO_Training/baggage_detection/datasets/shopping
Cloning YOLOv5 repository...
Cloning into 'yolov5'...
remote: Enumerating objects: 17270, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 17270 (delta 0), reused 0 (delta 0), pack-reused 17269 (from 2)
Receiving objects: 100% (17270/17270), 16.10 MiB | 15.04 MiB/s, done.
Resolving deltas: 100% (11855/11855), done.
Installing YOLOv5 requirements...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.4 MB/s e

In [17]:
# After training, run inference
!python yolo_inference.py \
  --model /content/drive/MyDrive/Edge_AI_CW/YOLO_Training/baggage_detection/models/merged/weights/best.pt \
  --dir test_images \
  --output /content/drive/MyDrive/Edge_AI_CW/YOLO_Training/baggage_detection/results

Error: Directory not found: test_images


In [18]:
# Export to .pb format
!python export_to_pb.py \
  --weights /content/drive/MyDrive/Edge_AI_CW/YOLO_Training/baggage_detection/models/merged/weights/best.pt \
  --output-dir /content/drive/MyDrive/Edge_AI_CW/YOLO_Training/baggage_detection/exported_models

2025-03-13 15:33:00.200920: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741879980.648813   82075 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741879980.773583   82075 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-13 15:33:01.750609: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Exporting model from /content/drive/MyDrive/Edge_AI_CW/YOLO_Training/baggage_detection/models/merged/weights/best.pt to Tenso